# **House Sales Price Study**

## Objectives

* Answer business requirement 1:
  * The client is interested to understand the most relevant house variables correlate against the sale price.

## Inputs

* outputs/datasets/collection/house_prices_after_inspection.csv

## Outputs

* Generate code that answers business requirement 1 and can be used to build the Streamlit App

## Additional Comments

* Data derives from Kaggle but has been provided by CI 


---

# Change working directory to the parent folder

Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load the Data

In [ ]:
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/house_prices_after_inspection.csv")
#df.head()
df.tail()

In [ ]:
df_to_clean = df.copy()
df_to_clean.tail()

# Create a profile report for quick Exploratory Data Analysis (EDA)

In [ ]:
from ydata_profiling import ProfileReport
profile_report= ProfileReport(df=df_to_clean, minimal=True)
#profile_report
#profile_report.to_notebook_iframe()

## EDA Observations

* This dataset hast a predominance for numerical variables.
* Only 4 variables are categorical: BsmtExposure, BsmtFinType1, GarageFinish, KitchenQual
* The 4 categorical variables are imbalanced.
* Several variables have missing values and zeros.
* Most numerical variables seem to be not normally distributed. 

# Handle Missing Values (NaN)

In [ ]:
df_to_clean.isnull().sum().to_frame(name="Is Null")

In [ ]:
categorical_variables = df_to_clean.select_dtypes(include='object').columns.to_list()
categorical_variables

In [ ]:
for col in df_to_clean.select_dtypes(include="object"):
    print(df_to_clean[col].value_counts())

In [ ]:
from feature_engine.imputation import CategoricalImputer
categorical_imputer= CategoricalImputer(imputation_method='missing',
                                                  fill_value='Missing',
                                                  variables=categorical_variables)
df_categ_imputed =categorical_imputer.fit_transform(df_to_clean)

In [ ]:
df_categ_imputed[categorical_variables].isnull().sum().to_frame(name="Is Null")

In [ ]:
import pingouin as pg
pg.normality(data=df_categ_imputed, alpha = 0.05) # check normality: They are all not normally distributed


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
# for col in df.select_dtypes(include="number").columns:
  # sns.histplot(data=df, x=col, kde=True)
  # plt.show()
  # print('\n')

numerical_variables = df_categ_imputed.select_dtypes(include="number").columns

n_cols = 3
n_rows = (len(numerical_variables) + n_cols - 1) // n_cols

# Create the figure and subplots grid
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))

# Flatten the axes array if there's more than one row/column
axes = axes.flatten()

# Iterate through columns and plot on respective axes
for i, col in enumerate(numerical_variables):
    sns.histplot(data=df_categ_imputed, x=col, kde=True, ax=axes[i]) 
    axes[i].set_title(f'Distribution of {col}')

# Remove unused subplots
for j in range(len(numerical_variables), len(axes)):
    fig.delaxes(axes[j])

# Prevent titles/labels from overlapping
plt.tight_layout()

# Display all plots
plt.show()


In [ ]:
numerical_variables = df_categ_imputed.select_dtypes(include="number").columns.to_list()

from feature_engine.imputation import MeanMedianImputer
numerical_imputer = MeanMedianImputer(imputation_method='median',
                            variables= numerical_variables)

df_categ_and_numb_imputed = numerical_imputer.fit_transform(df_categ_imputed)


In [ ]:
df_categ_and_numb_imputed.isnull().sum().to_frame(name="Is Null")

# Correlation Study: Pearson and Spearman

**Goal:** identify how the target (SalesPrice) correlate to the variables, and retrieve the top 5 correlation variables for SalesPrice.

* Step 1: Handle M

* Step 1: Since Spearman and Peason need numeric variables: transform categorical variables to numerical variables using one hot encoding.

In [ ]:
from feature_engine.encoding import OneHotEncoder
one_hot_encoder = OneHotEncoder(variables=df_categ_and_numb_imputed.select_dtypes(include='object').columns.to_list(), drop_last=False)
#one_hot_encoder = OneHotEncoder(variables=df.columns[df.dtypes=='object'].to_list(), drop_last=False)
df_ohe = one_hot_encoder.fit_transform(df_categ_and_numb_imputed)
df_ohe.tail()


In [ ]:
categorical_variables = df_ohe.select_dtypes(include='object').columns.to_list()
categorical_variables

In [ ]:
corr_pearson = df_ohe.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10).to_frame(name="Correlation Coefficient")
corr_pearson

---

In [ ]:
corr_spearman = df_ohe.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10).to_frame(name="Correlation Coefficient")
corr_spearman

Concanttenate them
vars_to study_
plot them 
parallel plot
Summary observations


# Load and Inspect Kaggle data

### Read CSV files

In [ ]:
import pandas as pd
df_house_prices = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv") 
df_house_prices.head()
# print(df.shape)


In [ ]:
df_inherited_houses = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv") 
df_inherited_houses.head()
# print(df.shape)

### Read TXT files

In [ ]:
df_house_metadata = pd.read_csv(f"inputs/datasets/raw/house-metadata.txt", header=None) 
df_house_metadata.head()
# print(df_house_metadata.shape)

### DataFrames Summary

In [ ]:
df_house_prices.info()

In [ ]:
df_inherited_houses.info()

In [ ]:
df_house_metadata.info()

### Check for duplicates 
* There are no duplicates in the data, there is also no unique indentifier such as "HouseID" to drop.

In [ ]:
df_house_prices.duplicated().sum()

### Confirm Target data type
* The target is already a numeric variable.

In [ ]:
df_house_prices['SalePrice'].dtype

### Notes
* The variables GarageYrBlt, YearBuilt and YearRemodAdd are numeric. 
* While they could be converted to datetime data type, their current numerical format facilitates their use in Pearson and Spearman correlation analyses and as direct inputs for the regression model.

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

### Create outputs directory

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/collection')
except Exception as e:
  print(e)


### Save the data under as csv

In [ ]:
df_house_prices.to_csv(f"outputs/datasets/collection/house_prices_after_inspection.csv", index=False)